In [122]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Formas de caracterizar en contexto

Haremos los siguientes experimentos para ver la mejor forma de caracterizar el contexto:
    - bolsa de palabras (unigramas, bigramas, subwords, con y sin stopwords)
    - Agregado por mi: tfidf con normalización porque los vectores son muy sparsos
    - si hay interacción del usuario objetivo y cómo se representa
    - si hay respuestas
    - si hay multimedia y de qué tipo
__COMPLETAR__

Usaremos diferentes word embeddings, obtenidos de diferentes tipos de corpus: genéricos o específicos de lenguaje generado por usuario o específicos del corpus del que estamos aprendiendo.
    - Es decir, tres diferentes tipos de word embeddings:
      - SBWCE
      - con el mismo corpus de chats
      - con un corpus de twitter

## Cargando los datos de entrenamiento

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('short_ans3.csv')

Acordandonos de los datos..

In [4]:
sum(df.context.isnull())

1

In [5]:
df

,idchat,context,label
0,0,NaN,https://m.facebook.com/story.php?story_fbid=14...
1,0,https://m.facebook.com/story.php?story_fbid=14...,nonDef
2,1,😂😂😂😂😂😂😂😂😂😂. Que es este lugar 👀. Qur hace este...,Bienvenido
3,1,Bienvenido. Vengo a pasarle fotos obscenas e i...,👀
4,1,👀. El mundo de nunca jamas?),😂😂
5,1,😂😂,😂😂😂
6,1,😂😂😂,Oh sii
7,1,Oh sii. Seremos niños por siempre,Quienes ? 👀😂
8,1,Quienes ? 👀😂,Yo
9,1,Yo. 😏,😏


# Pre procesamiento

Como habíamos dicho es mejor pre procesar las palabras

In [6]:
from string import punctuation
import re
import unicodedata

In [93]:
STOP_WORDS = list(punctuation)

def remove_repeted(text):
    return re.sub(r'(\w)\1*', r'\1', text)

def remove_sw(text):
    return re.sub('{1}{0}{1}'.format(punctuation + '¡', '[]'), '', text)

def dig2num(word):
    return re.sub('(\d+[\WxX]*[^ ]*)+', 'NUM', word)

def remove_tildes(cadena):
    s = ''.join((c for c in unicodedata.normalize('NFD',cadena) if unicodedata.category(c) != 'Mn'))
    return s

def remove_ok(text):
    return re.sub('(ok)+[a-zA-Z]*', 'ok', text)

def remove_ja(text):
    return re.sub('(ja[ajs]*)+', 'ja', text)

def clean_text(text):
    return dig2num(remove_tildes(remove_ok(remove_ja(text.lower()))))

def clean_text_1(text):
    return remove_repeted(clean_text(remove_sw(text)))

## Vectorizando

#### BOG: 1-gramas

In [117]:
vectorizer = CountVectorizer(min_df=10, preprocessor=clean_text_1, ngram_range=(1, 1))

In [118]:
representation = vectorizer.fit(df[1:].context)

Entonces obtenemos alrededor de tantas features:

In [121]:
print(representation.get_feature_names()[:100])
len(representation.get_feature_names())

['NUM', 'aNUM', 'abajo', 'abi', 'abierta', 'abiertas', 'abierto', 'abra', 'abran', 'abrazo', 'abrazos', 'abre', 'abren', 'abri', 'abril', 'abrir', 'abro', 'abuela', 'abuelo', 'aburido', 'abuso', 'ac', 'aca', 'acaba', 'acaban', 'acabar', 'acabaron', 'acabas', 'acabe', 'acabo', 'academia', 'acaso', 'aceder', 'acepta', 'acepte', 'acerca', 'aceso', 'acion', 'aclarar', 'aclaro', 'acompana', 'acompanar', 'acompano', 'acordaba', 'acordar', 'acordas', 'acordate', 'acorde', 'activa', 'activan', 'activar', 'active', 'actividad', 'actividades', 'activo', 'activos', 'acto', 'actores', 'actual', 'actualiza', 'actualizacion', 'actualizar', 'actualizo', 'acuerda', 'acuerdan', 'acuerdo', 'acv', 'ad', 'adelantar', 'adelante', 'adelanto', 'ademas', 'adentro', 'adios', 'adjuntado', 'admin', 'administrador', 'adri', 'adrian', 'aeropuerto', 'afecta', 'afiche', 'afuera', 'again', 'agara', 'agarar', 'agare', 'agaro', 'age', 'agenda', 'agendado', 'agendo', 'ago', 'agosto', 'agotada', 'agradece', 'agradeceria'

5204

#### BOG: 2-gramas

In [11]:
representation.set_params(ngram_range=(1, 2))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(1, 2),
        preprocessor=<function clean_text at 0x7f0c0be776a8>,
        stop_words=['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [12]:
len(representation.fit(df[1:].context).get_feature_names())

14206

#### BOG: 3-gramas

In [13]:
representation.set_params(ngram_range=(1, 3))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(1, 3),
        preprocessor=<function clean_text at 0x7f0c0be776a8>,
        stop_words=['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [14]:
len(representation.fit(df[1:].context).get_feature_names())

16925

In [15]:
vector = representation.transform(df[1:].context)

In [16]:
vector.get_shape()

(56974, 16925)

In [26]:
vector.getrow(56953).sum()

4

:o esto quiere decir que en 16925 columnas, 4 no son 0. Una matriz totalmente esparsa. Lo mejor sería implementar un word2vec o tfidf con normalización para ver que sucede.

#### BOG: (3,4)-Subwords

In [101]:
df[1:].context.map(clean_text_1).head(30)

1         htpsmfacebokNUM hola te confundiste de numero
2     😂😂😂😂😂😂😂😂😂😂 que es este lugar 👀 qur hace este a...
3     bienvenido vengo a pasarle fotos obscenas e ir...
4                             👀 el mundo de nunca jamas
5                                                    😂😂
6                                                   😂😂😂
7                       oh si seremos ninos por siempre
8                                           quienes  👀😂
9                                                  yo 😏
10                                                    😏
11                                                    😏
12     😏 ja eso si que es bastante creativo muy bueno 😄
13                                                    😎
14                                                  🌚 🌚
15                                                    😏
16                                                 hola
17    hola 🇻🇪 que yo me desaparezca no hay lio pero ...
18    😱😱 quien ja mira que adulador es nuestro m

In [110]:
vectorizer = CountVectorizer(min_df=10, stop_words=STOP_WORDS, preprocessor=clean_text_1, analyzer='char', ngram_range=(3,4))

In [111]:
vectorizer

CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(3, 4),
        preprocessor=<function clean_text_1 at 0x7f0c01940620>,
        stop_words=['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [112]:
representation = vectorizer.fit(df[1:].context)

In [116]:
len(representation.get_feature_names())

23187

### TFidf: 1-gramas

In [124]:
vectorizer = TfidfVectorizer(min_df=10, preprocessor=clean_text_1, ngram_range=(1, 1))

In [129]:
representation = vectorizer.fit(df[1:].context)

In [131]:
BOG_matrix = representation.transform(df[1:].context)